# Denoising Diffusion Probabilistic Models (DDPM)

This notebook is an implementation of the DDPM model proposed in the article by Ho et al. [1].

In [1]:
# PyTorch imports
import torch
import torchvision

## 0. Constants

## 1. Dataset

Let's use CelebA as our dataset.

In [5]:
# Each item in dataset is a tuple of (torch.Tensor, torch.Tensor).
# The first tensor is of shape (3, 218, 178) and dtype torch.float32, which is the image.
# The second tensor is of shape (40,) and dtype torch.int64, which is the attributes consist of 0 and 1.
dataset = torchvision.datasets.CelebA('./data', download=True, transform=torchvision.transforms.ToTensor())

Files already downloaded and verified


## References

[1] Jonathan Ho, Ajay Jain and Pieter Abbeel. Denoising Diffusion Probabilistic Models. arXiv, 2020.